# 🏠 625 Ocean Street — AI Architecture Rendering Pipeline
### ComfyUI + ControlNet on Free Google Colab GPU

**What this does:** Runs a full AI rendering studio in your browser using Google's free T4 GPU (16GB VRAM). Upload your floor plan → get photorealistic architectural renders.

**Your machine:** HP ProDesk 600 G1 — no GPU needed, everything runs on Google's servers.

---

### ⚡ Quick Start (3 steps)
1. **Run Cell 1** — Check GPU & install everything (~5-8 min)
2. **Run Cell 2** — Download models (~3-5 min)
3. **Run Cell 3** — Launch WebUI → click the public URL → start rendering

---

**⚠️ IMPORTANT:** Before running, go to **Runtime → Change runtime type → GPU (T4)** in the menu above.

*Built by BidDeed.AI / Everest Capital USA — Feb 2026*

In [ ]:
#@title ## 🔧 Step 0: Verify GPU (Run this first!)
#@markdown Confirms you have a GPU assigned. If it says "No GPU", go to Runtime → Change runtime type → GPU

import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
if result.returncode == 0:
    # Extract GPU name and memory
    lines = result.stdout.strip().split('\n')
    print("✅ GPU DETECTED!")
    print("=" * 50)
    for line in lines:
        if 'Tesla' in line or 'T4' in line or 'A100' in line or 'V100' in line or 'L4' in line:
            print(f"  GPU: {line.strip()}")
        if 'MiB' in line and '/' in line:
            print(f"  VRAM: {line.strip()}")
    print("=" * 50)
    print("\n🚀 You're good to go! Run the next cell.")
else:
    print("❌ NO GPU DETECTED!")
    print("Go to: Runtime → Change runtime type → Hardware accelerator → GPU (T4)")
    print("Then re-run this cell.")

In [ ]:
#@title ## ⚙️ Step 1: Install Stable Diffusion WebUI + ControlNet (~5-8 min)
#@markdown This installs A1111 Stable Diffusion WebUI (Forge fork) with ControlNet extension.
#@markdown Forge = faster rendering + better ControlNet support, built by ControlNet's creator.

import os
from pathlib import Path

# ============================================
# CONFIGURATION
# ============================================
INSTALL_DIR = '/content/stable-diffusion-webui-forge'
USE_FORGE = True  # Forge is recommended

print("🔧 Installing Stable Diffusion WebUI (Forge)...")
print("=" * 50)

# Install system dependencies
print("\n📦 [1/5] Installing system packages...")
!apt-get -y install -qq aria2 > /dev/null 2>&1
print("  ✅ System packages ready")

# Clone Forge
if not os.path.exists(INSTALL_DIR):
    print("\n📥 [2/5] Cloning Forge repository...")
    !git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git {INSTALL_DIR} 2>&1 | tail -3
    print("  ✅ Forge cloned")
else:
    print("\n📁 [2/5] Forge already installed, updating...")
    !cd {INSTALL_DIR} && git pull 2>&1 | tail -3
    print("  ✅ Forge updated")

# Install ControlNet extension
CONTROLNET_DIR = f"{INSTALL_DIR}/extensions/sd-webui-controlnet"
if not os.path.exists(CONTROLNET_DIR):
    print("\n🎛️ [3/5] Installing ControlNet extension...")
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git {CONTROLNET_DIR} 2>&1 | tail -3
    print("  ✅ ControlNet extension installed")
else:
    print("\n🎛️ [3/5] ControlNet already installed, updating...")
    !cd {CONTROLNET_DIR} && git pull 2>&1 | tail -3
    print("  ✅ ControlNet updated")

# Install additional extensions for better results
ADETAILER_DIR = f"{INSTALL_DIR}/extensions/adetailer"
if not os.path.exists(ADETAILER_DIR):
    print("\n🔍 [4/5] Installing ADetailer (auto-fix faces/details)...")
    !git clone https://github.com/Bing-su/adetailer.git {ADETAILER_DIR} 2>&1 | tail -3
    print("  ✅ ADetailer installed")
else:
    print("\n🔍 [4/5] ADetailer already installed")

# Create necessary directories
print("\n📁 [5/5] Creating model directories...")
dirs = [
    f"{INSTALL_DIR}/models/Stable-diffusion",
    f"{INSTALL_DIR}/models/ControlNet",
    f"{INSTALL_DIR}/models/ESRGAN",
    f"{INSTALL_DIR}/outputs",
    f"{CONTROLNET_DIR}/models",
]
for d in dirs:
    os.makedirs(d, exist_ok=True)
print("  ✅ Directories ready")

print("\n" + "=" * 50)
print("✅ INSTALLATION COMPLETE!")
print("\n👉 Now run Step 2 to download the AI models.")

In [ ]:
#@title ## 📦 Step 2: Download AI Models (~3-5 min)
#@markdown Downloads the base Stable Diffusion model + ControlNet models for architecture rendering.
#@markdown Using aria2 for fast parallel downloads.

INSTALL_DIR = '/content/stable-diffusion-webui-forge'
CONTROLNET_DIR = f"{INSTALL_DIR}/extensions/sd-webui-controlnet"

print("📦 Downloading AI Models...")
print("=" * 50)

# ============================================
# BASE MODEL: Realistic Vision v6.0 B1
# Best general-purpose photorealistic model
# ============================================
model_path = f"{INSTALL_DIR}/models/Stable-diffusion/realisticVisionV60B1_v60B1VAE.safetensors"
if not os.path.exists(model_path):
    print("\n🎨 [1/5] Downloading Realistic Vision v6.0 (~2GB)...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
        "https://civitai.com/api/download/models/245598?type=Model&format=SafeTensor&size=pruned&fp=fp16" \
        -d "{INSTALL_DIR}/models/Stable-diffusion" \
        -o "realisticVisionV60B1_v60B1VAE.safetensors" 2>&1 | tail -5
    print("  ✅ Realistic Vision v6.0 downloaded")
else:
    print("\n🎨 [1/5] Realistic Vision v6.0 already downloaded ✅")

# ============================================
# CONTROLNET: MLSD (Straight Line Detection)
# Critical for floor plan → render
# ============================================
mlsd_path = f"{CONTROLNET_DIR}/models/control_v11p_sd15_mlsd.pth"
if not os.path.exists(mlsd_path):
    print("\n📐 [2/5] Downloading ControlNet MLSD — Straight Lines (~1.4GB)...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
        "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth" \
        -d "{CONTROLNET_DIR}/models" \
        -o "control_v11p_sd15_mlsd.pth" 2>&1 | tail -5
    print("  ✅ ControlNet MLSD downloaded (floor plan lines)")
else:
    print("\n📐 [2/5] ControlNet MLSD already downloaded ✅")

# ============================================
# CONTROLNET: Depth (MiDaS Depth Map)
# For spatial relationships / 3D perspective
# ============================================
depth_path = f"{CONTROLNET_DIR}/models/control_v11f1p_sd15_depth.pth"
if not os.path.exists(depth_path):
    print("\n🗺️ [3/5] Downloading ControlNet Depth — Spatial (~1.4GB)...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
        "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth" \
        -d "{CONTROLNET_DIR}/models" \
        -o "control_v11f1p_sd15_depth.pth" 2>&1 | tail -5
    print("  ✅ ControlNet Depth downloaded (spatial relationships)")
else:
    print("\n🗺️ [3/5] ControlNet Depth already downloaded ✅")

# ============================================
# CONTROLNET: Canny (Edge Detection)
# For preserving outlines
# ============================================
canny_path = f"{CONTROLNET_DIR}/models/control_v11p_sd15_canny.pth"
if not os.path.exists(canny_path):
    print("\n✏️ [4/5] Downloading ControlNet Canny — Edge Detection (~1.4GB)...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
        "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth" \
        -d "{CONTROLNET_DIR}/models" \
        -o "control_v11p_sd15_canny.pth" 2>&1 | tail -5
    print("  ✅ ControlNet Canny downloaded (edge outlines)")
else:
    print("\n✏️ [4/5] ControlNet Canny already downloaded ✅")

# ============================================
# UPSCALER: 4x-UltraSharp
# For upscaling renders to 4K
# ============================================
upscaler_path = f"{INSTALL_DIR}/models/ESRGAN/4x-UltraSharp.pth"
if not os.path.exists(upscaler_path):
    print("\n🔍 [5/5] Downloading 4x-UltraSharp upscaler (~67MB)...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
        "https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x-UltraSharp.pth" \
        -d "{INSTALL_DIR}/models/ESRGAN" \
        -o "4x-UltraSharp.pth" 2>&1 | tail -5
    print("  ✅ 4x-UltraSharp upscaler downloaded")
else:
    print("\n🔍 [5/5] 4x-UltraSharp already downloaded ✅")

# Verify all files
print("\n" + "=" * 50)
print("📋 MODEL VERIFICATION:")
import os
checks = [
    ("Realistic Vision v6.0", model_path),
    ("ControlNet MLSD", mlsd_path),
    ("ControlNet Depth", depth_path),
    ("ControlNet Canny", canny_path),
    ("4x-UltraSharp", upscaler_path),
]
all_good = True
for name, path in checks:
    exists = os.path.exists(path)
    size = os.path.getsize(path) / (1024**3) if exists else 0
    status = f"✅ {size:.1f}GB" if exists and size > 0.01 else "❌ MISSING"
    print(f"  {name}: {status}")
    if not exists:
        all_good = False

print("=" * 50)
if all_good:
    print("\n✅ ALL MODELS READY!")
    print("\n👉 Now run Step 3 to launch the WebUI.")
else:
    print("\n⚠️ Some models missing. Re-run this cell to retry downloads.")

In [ ]:
#@title ## 🚀 Step 3: Launch WebUI (Click the gradio.live link!)
#@markdown This starts the Stable Diffusion WebUI with a public URL you can access from any browser.
#@markdown **Look for the `https://xxxxx.gradio.live` link** in the output below — click it to open.

INSTALL_DIR = '/content/stable-diffusion-webui-forge'

print("🚀 Launching Stable Diffusion WebUI...")
print("=" * 50)
print("⏳ First launch takes 2-3 minutes to initialize.")
print("📌 Look for the PUBLIC URL (gradio.live) below!")
print("=" * 50)
print("")

%cd {INSTALL_DIR}

!python launch.py \
    --share \
    --xformers \
    --enable-insecure-extension-access \
    --no-half-vae \
    --theme dark \
    --controlnet-dir extensions/sd-webui-controlnet/models \
    --opt-sdp-attention 2>&1 | tee /content/webui.log

---
## 🎯 How to Render 625 Ocean Street

Once the WebUI opens in your browser:

### Quick Render Steps:
1. **Select model**: Choose `realisticVisionV60B1_v60B1VAE` from the checkpoint dropdown (top left)
2. **Paste prompt**: Copy a prompt from the Prompt Library below into the positive prompt box
3. **Paste negative prompt**: Copy the negative prompt into the negative prompt box
4. **Enable ControlNet**: Scroll down → expand ControlNet → check **Enable**
5. **Upload floor plan**: Drop your 625 Ocean Street floor plan image into the ControlNet image area
6. **Set ControlNet model**: Preprocessor = `mlsd` | Model = `control_v11p_sd15_mlsd`
7. **Set generation size**: Width = 1024, Height = 768 (landscape for exteriors)
8. **Set sampler**: `DPM++ 2M SDE Karras` | Steps: 30 | CFG Scale: 7.5
9. **Click Generate!** → Wait 30-60 seconds

### ControlNet Settings:
| Setting | Value |
|---------|-------|
| Preprocessor | `mlsd` (exteriors) or `depth_midas` (interiors) |
| Model | `control_v11p_sd15_mlsd` or `control_v11f1p_sd15_depth` |
| Control Weight | 0.85 |
| Starting Control Step | 0.0 |
| Ending Control Step | 0.95 |

---

## ✍️ 625 Ocean Street — Prompt Library

Copy these directly into the WebUI prompt boxes.

---

### 🏠 EXTERIOR — Golden Hour Hero Shot
```
modern luxury coastal home, 625 Ocean Street Satellite Beach Florida, two-story oceanfront property, rooftop pool deck with infinity edge pool, contemporary architecture, white stucco exterior with natural stone accents, floor-to-ceiling glass windows, palm trees, tropical landscaping, golden hour warm lighting, photorealistic architectural rendering, 8k, ultra detailed, professional architectural photography, blue sky with scattered clouds, lush green lawn
```

### 🌅 EXTERIOR — Twilight Dramatic
```
luxury modern beach house, architectural exterior photograph, twilight blue hour, interior lights glowing warm through large windows, two-story contemporary design, rooftop pool with ambient LED lighting, fire pit on deck, palm trees silhouetted against sunset sky, Satellite Beach Florida oceanfront, professional real estate photography, long exposure, dramatic sky, 8k ultra detailed
```

### 🛋️ INTERIOR — Open Living Space
```
modern luxury home interior, open concept living room and kitchen, floor-to-ceiling ocean view windows, polished concrete floors, white walls with warm wood accents, designer furniture, large kitchen island with waterfall countertop, recessed lighting and pendant lights, coastal contemporary style, natural light flooding in, professional interior architecture photography, 8k, ultra detailed, warm tones
```

### 🏊 POOL DECK — Daytime
```
luxury rooftop pool deck, infinity edge swimming pool with integrated spa, ocean panoramic view, modern fire pit lounge area, outdoor dining space with pergola, built-in BBQ kitchen, premium deck tile flooring, glass railing with ocean view, tropical plants and palms, Satellite Beach Florida, bright sunny day, professional architectural photography, 8k ultra detailed, resort-style luxury
```

### 🌙 POOL DECK — Evening
```
rooftop pool deck at night, illuminated infinity pool with underwater LED lights, fire pit with dancing flames, string lights overhead, ocean in background under starry sky, modern outdoor furniture, warm ambient lighting, luxury resort atmosphere, Satellite Beach Florida rooftop, professional nighttime architectural photography, long exposure, 8k
```

### 🛏️ INTERIOR — Master Bedroom
```
luxury master bedroom suite, ocean view through panoramic windows, king size bed with premium linens, modern minimalist design, en-suite bathroom visible, walk-in closet, soft ambient lighting, neutral color palette with blue accents, hardwood floors, custom built-in cabinetry, professional interior design photography, 8k, peaceful morning light
```

### 🦅 EXTERIOR — Aerial Drone Shot
```
aerial drone photograph of modern luxury coastal home, bird's eye view, rooftop pool deck clearly visible, infinity pool overlooking ocean, two-story flat roof contemporary architecture, circular driveway, palm tree lined property, Satellite Beach Florida, Atlantic Ocean in background, professional drone real estate photography, 8k, sunny day, sharp detail
```

---

### ❌ NEGATIVE PROMPT (Use for ALL renders)
```
blurry, low quality, distorted, ugly, deformed, cartoon, anime, sketch, drawing, painting, watermark, text, signature, low resolution, oversaturated, unrealistic proportions, bad architecture, impossible geometry, floating objects, extra limbs, disfigured, mutation, poorly drawn, bad anatomy, extra fingers, fewer digits, cropped, worst quality, jpeg artifacts
```

---
## 💡 Pro Tips

### Iterating on renders:
- **Lock the seed**: When you get a render you like, note the seed number → use it to make small prompt changes while keeping the same composition
- **Batch generate**: Set Batch Count to 4 → generates 4 variations → pick the best
- **Inpainting**: Switch to img2img tab → Inpaint → mask areas you want to change (e.g., swap window styles, change landscaping)

### ControlNet tips:
- **MLSD** = best for floor plans with clear straight lines (walls, doors)
- **Depth** = best for 3D views / perspective shots from your Three.js model
- **Canny** = best for hand-drawn sketches or photos of existing buildings
- **Lower weight (0.6-0.7)** = more creative freedom, less strict to floor plan
- **Higher weight (0.9-1.0)** = more faithful to floor plan, less artistic freedom

### Saving your work:
- All renders save to the `outputs/` folder in Colab
- **Download renders**: Click the download button under each generated image
- **⚠️ Colab sessions expire** after ~12 hours — download your renders before closing!

### Upscaling to 4K:
- Go to **Extras** tab → upload your best render → select **4x-UltraSharp** → Scale to 4x → Generate
- A 1024×768 render becomes 4096×3072 (4K+)

---
*625 Ocean Street Rendering Pipeline • BidDeed.AI / Everest Capital USA • Feb 2026*

In [ ]:
#@title ## 📥 Download All Renders (Run before closing Colab!)
#@markdown Zips all generated images for download. Run this before your session expires.

import os
import shutil
from google.colab import files

INSTALL_DIR = '/content/stable-diffusion-webui-forge'
OUTPUT_DIR = f'{INSTALL_DIR}/outputs'
ZIP_NAME = '625_Ocean_Street_Renders'

# Count files
total_files = 0
for root, dirs, filenames in os.walk(OUTPUT_DIR):
    for f in filenames:
        if f.endswith(('.png', '.jpg', '.jpeg', '.webp')):
            total_files += 1

if total_files == 0:
    print("❌ No renders found yet. Generate some images first!")
else:
    print(f"📦 Found {total_files} renders. Creating zip...")
    shutil.make_archive(f'/content/{ZIP_NAME}', 'zip', OUTPUT_DIR)
    zip_size = os.path.getsize(f'/content/{ZIP_NAME}.zip') / (1024**2)
    print(f"✅ Created {ZIP_NAME}.zip ({zip_size:.1f}MB)")
    print("\n📥 Download starting...")
    files.download(f'/content/{ZIP_NAME}.zip')
    print("\n✅ Download complete! Check your Downloads folder.")